두 엑셀 파일, `game_Ids.xlsx`와 `crawling_result_with_codenames.xlsx`를 비교하여 팀 이름의 약칭을 수정하고 추출한 후, 그 결과를 `Abbreviated_name_extraction.xlsx` 파일로 저장합니다. 이 과정은 블루팀과 레드팀 모두에 대해 수행됩니다.

In [46]:
import pandas as pd

# 데이터 불러오기
game_ids = pd.read_excel('game_Ids.xlsx')
crawling = pd.read_excel('crawling_result_with_codenames.xlsx')

# 블루팀, 레드팀 코드 만 가져오기
game_ids_blue = game_ids[['blueteam_name', 'blueteam_code']]
game_ids_red = game_ids[['redteam_name', 'redteam_code']]
crawling_blue = crawling[['blueteam', 'blueteam_code']]
crawling_red = crawling[['redteam', 'redteam_code']]

# 블루팀, 레드팀 이름이 같은 행만 선택
same_teams_blue = pd.merge(game_ids_blue, crawling_blue, left_on='blueteam_name', right_on='blueteam', how='inner', suffixes=('_game_ids', '_crawling'))
same_teams_red = pd.merge(game_ids_red, crawling_red, left_on='redteam_name', right_on='redteam', how='inner', suffixes=('_game_ids', '_crawling'))

# 중복된 행 제거
same_teams_blue = same_teams_blue.drop_duplicates()
same_teams_red = same_teams_red.drop_duplicates()

# 결과를 엑셀 파일로 저장
result_blue = same_teams_blue[['blueteam_name', 'blueteam_code_crawling', 'blueteam_code_game_ids']].rename(columns={'blueteam_name': 'teamname', 'blueteam_code_crawling': 'crawling', 'blueteam_code_game_ids': 'game_ids'})
result_red = same_teams_red[['redteam_name', 'redteam_code_crawling', 'redteam_code_game_ids']].rename(columns={'redteam_name': 'teamname', 'redteam_code_crawling': 'crawling', 'redteam_code_game_ids': 'game_ids'})

# 두 DataFrame을 하나로 합치기
result = pd.concat([result_blue, result_red])

# 결과를 엑셀 파일로 저장
result.to_excel('Abbreviated_name_extraction.xlsx', index=False)


`Abbreviated_name_extraction.xlsx` 파일과 `team_code.json` 파일을 비교하여, 팀 코드가 잘못된 경우 정상 값으로 수정하고, 그 결과를 `team_code.json` 파일에 다시 저장합니다. 각 팀 코드는 새로운 줄에 위치하게 됩니다. 이렇게 하면, `team_code.json` 파일의 팀 코드 정보가 업데이트되고 깔끔하게 정리됩니다.

In [51]:
import pandas as pd
import json

# result.xlsx 파일 불러오기
result = pd.read_excel('Abbreviated_name_extraction.xlsx')

# team_code.json 파일 불러오기
with open('team_code.json', 'r') as f:
    team_codes = json.load(f)

# result.xlsx 파일에 있는 각 팀의 이름과 그에 해당하는 팀 코드를 확인하고,
# team_code.json 파일에 있는 팀 코드가 잘못된 경우, 정상 값으로 바꾸기
for i, row in result.iterrows():
    team_name = row['teamname']
    correct_team_code = row['game_ids']
    
    if team_codes.get(team_name) != correct_team_code:
        team_codes[team_name] = correct_team_code

# 수정된 팀 코드를 team_code.json 파일로 다시 저장
# 각 팀 코드가 새로운 줄에 오도록 정렬
with open('team_code.json', 'w') as f:
    f.write(json.dumps(team_codes, indent=4))
